# Notebook 8 — *Scoring* y exportación para BI (FraSoHome)

**Objetivo (formativo):** aplicar los modelos entrenados en el Notebook 7 sobre datasets *ML-ready* (salida del Notebook 6), generar ficheros de *scoring* para BI (PowerBI/Tableau) y crear explicaciones simples (drivers) para entender por qué un cliente/producto aparece con un score alto.

> Nota: este notebook está diseñado para ser **robusto** con datos “sucios” (errores intencionales del caso) y con diferentes nombres de archivos. Si falta algún fichero, el notebook continúa y deja trazas claras de lo que no se pudo ejecutar.

---

## Entradas esperadas

Carpetas típicas (si seguiste los notebooks 1–7):
- `output_ml/`  
  - `FraSoHome_clientes_ML_ready.csv`  
  - `FraSoHome_propension_ML_ready.csv` *(si aplica)*  
- `output_models/`  
  - modelos `.joblib` guardados en el Notebook 7 (por ejemplo `best_churn_model.joblib`, `best_propension_model.joblib`)
- `output_features/` *(opcional para enriquecer con IDs/atributos legibles)*  
  - `features_clientes.csv`  
  - `dataset_propension_snapshots.csv`

## Salidas

- `output_scoring/` con:
  - `churn_scoring.csv` (si hay modelo de churn)
  - `propension_scoring.csv` (si hay modelo de propensión)
  - `*_global_drivers_top.csv` (top variables globales)
  - `scoring_actions_dictionary.csv` (diccionario de acción por decil)

In [ ]:
# =========================
# 0) Imports y configuración
# =========================
from __future__ import annotations

import os
import re
import json
import glob
import math
import datetime as dt
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd

# Intentamos importar joblib. Si no está, usamos pickle como fallback.
try:
    import joblib
    HAS_JOBLIB = True
except Exception:
    import pickle
    HAS_JOBLIB = False

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)

BASE_PATH = Path(".")  # Ajusta si ejecutas el notebook desde otro directorio
OUTPUT_SCORING_DIR = BASE_PATH / "output_scoring"
OUTPUT_SCORING_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_MODELS_DIR = BASE_PATH / "output_models"
OUTPUT_ML_DIR = BASE_PATH / "output_ml"
OUTPUT_FEATURES_DIR = BASE_PATH / "output_features"

SCORING_DATE = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("HAS_JOBLIB:", HAS_JOBLIB)
print("BASE_PATH:", BASE_PATH.resolve())
print("OUTPUT_SCORING_DIR:", OUTPUT_SCORING_DIR.resolve())

## 1) Funciones reutilizables

La idea es **encapsular la lógica** para que puedas:
- reutilizar funciones con cualquier dataframe (parámetros),
- testear piezas (lectura, alineación de columnas, scoring),
- modificar reglas sin romper todo el notebook.

In [ ]:
# =========================
# 1) Funciones reutilizables
# =========================

def safe_read_csv(path: Union[str, Path], dtype: str = "str", encoding: str = "utf-8") -> pd.DataFrame:
    """Lee un CSV en modo robusto (formativo).
    - dtype=str para no “romper” con errores de tipos.
    - keep_default_na=False para mantener valores como '' y strings raros si fuese necesario.
    """
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"No existe el fichero: {path}")
    return pd.read_csv(path, dtype=dtype, encoding=encoding, keep_default_na=False)

def find_latest_file(patterns: List[str], folder: Union[str, Path]) -> Optional[Path]:
    """Busca el archivo más reciente (por mtime) dentro de una carpeta, dado un conjunto de patrones glob."""
    folder = Path(folder)
    candidates: List[Path] = []
    for pat in patterns:
        candidates.extend([Path(p) for p in glob.glob(str(folder / pat))])
    candidates = [p for p in candidates if p.exists()]
    if not candidates:
        return None
    candidates.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    return candidates[0]

def load_model(path: Union[str, Path]):
    """Carga un modelo guardado. Soporta joblib o pickle."""
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"No existe el modelo: {path}")
    if HAS_JOBLIB:
        return joblib.load(path)
    else:
        with open(path, "rb") as f:
            return pickle.load(f)

def detect_target_column(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    """Devuelve la primera columna encontrada de una lista de candidatos."""
    cols = set(df.columns)
    for c in candidates:
        if c in cols:
            return c
    return None

def split_X_y(
    df: pd.DataFrame,
    target_candidates: List[str],
) -> Tuple[pd.DataFrame, Optional[pd.Series], Optional[str]]:
    """Separa X e y si encuentra un target.
    Devuelve (X, y_or_None, target_name_or_None)
    """
    target = detect_target_column(df, target_candidates)
    if target is None:
        return df.copy(), None, None
    y = df[target].copy()
    X = df.drop(columns=[target]).copy()
    return X, y, target

def coerce_numeric_df(X: pd.DataFrame) -> pd.DataFrame:
    """Convierte todas las columnas a numérico si es posible.
    Asume que el dataset ML-ready debería ser totalmente numérico (excepto IDs).
    """
    X2 = X.copy()
    for c in X2.columns:
        if X2[c].dtype == object:
            # Intento suave: reemplazo de coma decimal por punto si parece número
            s = X2[c].astype(str).str.strip()
            s = s.str.replace("€", "", regex=False).str.replace("EUR", "", regex=False)
            # caso coma decimal: 12,34 -> 12.34 (pero cuidado con miles, aquí asumimos ML-ready)
            s = s.str.replace(",", ".", regex=False)
            X2[c] = pd.to_numeric(s, errors="ignore")
    # Si quedan objetos, intentamos forzar a NaN salvo IDs
    return X2

def identify_id_like_columns(df: pd.DataFrame) -> List[str]:
    """Heurística para detectar columnas tipo ID/metadata que NO deberían entrar al modelo
    (si entrenaste sin ellas). Aun así, si el modelo espera esas columnas, las alinearemos después.
    """
    id_cols = []
    for c in df.columns:
        lc = c.lower()
        if lc in {"customer_id", "product_id", "order_id", "ticket_id", "store_id", "snapshot_date"}:
            id_cols.append(c); continue
        if lc.endswith("_id"):
            id_cols.append(c); continue
        if lc in {"scoring_date", "model_name"}:
            id_cols.append(c); continue
    return id_cols

def align_columns_to_feature_names(X: pd.DataFrame, feature_names: List[str]) -> pd.DataFrame:
    """Alinea X a una lista de nombres de features (añade faltantes con 0, elimina extras).
    Esto es CRÍTICO para scoring consistente.
    """
    X_aligned = X.copy()
    missing = [c for c in feature_names if c not in X_aligned.columns]
    extra = [c for c in X_aligned.columns if c not in feature_names]
    for c in missing:
        X_aligned[c] = 0.0
    if extra:
        X_aligned = X_aligned.drop(columns=extra)
    # reorden
    X_aligned = X_aligned[feature_names]
    return X_aligned

def predict_proba_safe(model, X: pd.DataFrame) -> np.ndarray:
    """Devuelve score de clase positiva.
    - Si existe predict_proba: usa [:, 1]
    - Si no existe, usa decision_function y lo pasa por sigmoide
    - Si nada existe, usa predict y lo trata como {0,1}
    """
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X)
        if proba.ndim == 2 and proba.shape[1] >= 2:
            return proba[:, 1]
        # caso raro: solo una columna
        return proba.ravel()
    if hasattr(model, "decision_function"):
        z = model.decision_function(X)
        # Sigmoide
        return 1.0 / (1.0 + np.exp(-z))
    # fallback
    pred = model.predict(X)
    return np.asarray(pred).astype(float)

def compute_deciles(scores: pd.Series, n: int = 10) -> pd.Series:
    """Crea deciles (1..n). 1=menor score, n=mayor score."""
    s = pd.Series(scores).astype(float)
    # qcut puede fallar si hay muchos valores iguales; duplicates='drop' reduce bins.
    try:
        return pd.qcut(s, q=n, labels=False, duplicates="drop") + 1
    except Exception:
        # fallback: ranking
        r = s.rank(method="average", pct=True)
        return (np.ceil(r * n)).clip(1, n).astype(int)

def get_global_feature_importance(model, feature_names: List[str], top_n: int = 20) -> pd.DataFrame:
    """Extrae importancia global si el modelo la expone.
    - Árboles: feature_importances_
    - Lineales: abs(coef_)
    """
    if hasattr(model, "feature_importances_"):
        imp = np.asarray(model.feature_importances_).ravel()
    elif hasattr(model, "coef_"):
        coef = np.asarray(model.coef_).ravel()
        imp = np.abs(coef)
    else:
        return pd.DataFrame(columns=["feature", "importance"])
    df_imp = pd.DataFrame({"feature": feature_names, "importance": imp})
    df_imp = df_imp.sort_values("importance", ascending=False).head(top_n).reset_index(drop=True)
    return df_imp

def explain_linear_row(model, x_row: pd.Series, feature_names: List[str], top_n: int = 5) -> pd.DataFrame:
    """Explicación local sencilla para modelos lineales: contribución = coef * valor.
    Útil para formación (no sustituye SHAP).
    """
    if not hasattr(model, "coef_"):
        return pd.DataFrame(columns=["feature", "value", "contribution"])
    coef = np.asarray(model.coef_).ravel()
    x = x_row.reindex(feature_names).astype(float).values
    contrib = coef * x
    df = pd.DataFrame({"feature": feature_names, "value": x, "contribution": contrib})
    df["abs_contribution"] = df["contribution"].abs()
    df = df.sort_values("abs_contribution", ascending=False).head(top_n).drop(columns=["abs_contribution"])
    return df.reset_index(drop=True)

def save_csv(df: pd.DataFrame, path: Union[str, Path]) -> None:
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(path, index=False, encoding="utf-8")
    print(f"✅ Guardado: {path}  | filas={len(df):,} cols={df.shape[1]}")

def print_head(df: pd.DataFrame, n: int = 5, title: str = ""):
    if title:
        print("\n" + "="*len(title))
        print(title)
        print("="*len(title))
    display(df.head(n))

## 2) Carga de modelos y datasets ML-ready

Convención (recomendada):
- churn: `best_churn_model.joblib` + `FraSoHome_clientes_ML_ready.csv`
- propensión: `best_propension_model.joblib` + `FraSoHome_propension_ML_ready.csv`

Si los nombres difieren, buscamos por patrones.

In [ ]:
# =========================
# 2) Localizar y cargar modelos/datasets
# =========================

# Patrones típicos (ajusta si tus nombres son distintos)
CHURN_MODEL_PATTERNS = [
    "*churn*model*.joblib",
    "best_churn*.joblib",
    "*churn*.joblib",
]

PROP_MODEL_PATTERNS = [
    "*prop*model*.joblib",
    "best_prop*.joblib",
    "*propension*.joblib",
    "*propensity*.joblib",
]

churn_model_path = find_latest_file(CHURN_MODEL_PATTERNS, OUTPUT_MODELS_DIR)
prop_model_path = find_latest_file(PROP_MODEL_PATTERNS, OUTPUT_MODELS_DIR)

print("churn_model_path:", churn_model_path)
print("prop_model_path :", prop_model_path)

# Datasets ML-ready
clientes_ml_path = (OUTPUT_ML_DIR / "FraSoHome_clientes_ML_ready.csv")
prop_ml_path = (OUTPUT_ML_DIR / "FraSoHome_propension_ML_ready.csv")

print("clientes_ml_path exists:", clientes_ml_path.exists(), clientes_ml_path)
print("prop_ml_path exists    :", prop_ml_path.exists(), prop_ml_path)

## 3) Scoring — Churn (clientes)

### Salida para BI
Generaremos un fichero con:
- `customer_id` (si se puede recuperar)
- `score_churn` (0–1)
- `pred_churn` con un umbral configurable (por defecto 0.5)
- `decil_churn` (1..10)
- `top_drivers` (si el modelo es lineal, explicación local sencilla)

In [ ]:
# =========================
# 3) Scoring churn (clientes)
# =========================

CHURN_TARGET_CANDIDATES = ["label_churn_180d", "churn", "target", "y"]

def score_dataset(
    df_ml_ready: pd.DataFrame,
    model_obj,
    target_candidates: List[str],
    id_cols_prefer: Optional[List[str]] = None,
    threshold: float = 0.5,
    model_name: str = "model",
    top_n_drivers: int = 3,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Scoring genérico:
    - soporta que el 'model_obj' sea:
        a) estimator sklearn
        b) dict {'model': estimator, 'feature_names': [...]}
    Devuelve:
      - scoring_df (BI-friendly)
      - global_drivers_df (importancia global)
    """
    # 1) Desempaquetar modelo si viene en dict
    feature_names = None
    model = model_obj
    if isinstance(model_obj, dict):
        model = model_obj.get("model", model_obj)
        feature_names = model_obj.get("feature_names") or model_obj.get("columns")  # alias
        if feature_names is not None:
            feature_names = list(feature_names)

    # 2) Separar X/y si existe target
    X_raw, y, y_name = split_X_y(df_ml_ready, target_candidates)

    # 3) Identificadores (para BI)
    id_cols = []
    if id_cols_prefer:
        id_cols = [c for c in id_cols_prefer if c in X_raw.columns]
    # Si no se especifican, intentamos heurística
    if not id_cols:
        id_cols = identify_id_like_columns(X_raw)
    id_df = X_raw[id_cols].copy() if id_cols else pd.DataFrame(index=X_raw.index)

    # 4) Datos para el modelo
    X = X_raw.drop(columns=id_cols, errors="ignore").copy()
    X = coerce_numeric_df(X)

    # 5) Alinear columnas si tenemos feature_names
    if feature_names is not None:
        X = align_columns_to_feature_names(X, feature_names)

    # 6) Convertir no-numéricos a NaN (y luego a 0 para scoring básico)
    #    (en un pipeline real se usaría el mismo preprocesador del entrenamiento)
    for c in X.columns:
        if not np.issubdtype(X[c].dtype, np.number):
            X[c] = pd.to_numeric(X[c], errors="coerce")
    X = X.fillna(0.0)

    # 7) Score
    scores = predict_proba_safe(model, X)
    scores = np.clip(scores, 0, 1)
    scores_s = pd.Series(scores, index=X.index, name="score")

    # 8) Ensamblar salida BI
    out = pd.DataFrame(index=X.index)
    if not id_df.empty:
        out = pd.concat([id_df.reset_index(drop=True), out.reset_index(drop=True)], axis=1)
    out["score"] = scores_s.reset_index(drop=True)
    out["pred"] = (out["score"] >= threshold).astype(int)
    out["decil"] = compute_deciles(out["score"], n=10)
    out["model_name"] = model_name
    out["scoring_date"] = SCORING_DATE
    if y is not None:
        # intentamos normalizar y a 0/1
        y_num = pd.to_numeric(y, errors="coerce").fillna(0).astype(int)
        out["y_true"] = y_num.reset_index(drop=True)

    # 9) Drivers globales
    feat_names_for_imp = feature_names if feature_names is not None else list(X.columns)
    global_imp = get_global_feature_importance(model, feat_names_for_imp, top_n=25)
    global_imp["model_name"] = model_name

    # 10) Drivers locales (solo para lineales)
    if hasattr(model, "coef_") and top_n_drivers > 0:
        # calcula top drivers para las N filas con score más alto
        top_idx = out["score"].sort_values(ascending=False).head(20).index
        driver_cols = []
        for i in top_idx:
            expl = explain_linear_row(model, X.loc[i], feat_names_for_imp, top_n=top_n_drivers)
            # convertimos a string para BI
            driver_str = "; ".join([f"{r['feature']}({r['contribution']:+.3f})" for _, r in expl.iterrows()])
            driver_cols.append((i, driver_str))
        driver_map = dict(driver_cols)
        out["top_drivers_linear"] = out.index.map(lambda i: driver_map.get(i, ""))

    return out, global_imp

# Ejecutar churn scoring si hay modelo y dataset
churn_scoring_df = None
churn_global_drivers = None

if churn_model_path and clientes_ml_path.exists():
    churn_model_obj = load_model(churn_model_path)
    clientes_ml = safe_read_csv(clientes_ml_path, dtype="str")

    # Intento: recuperar customer_id si está en features_clientes (opcional)
    # Si el ML-ready no lo contiene, no pasa nada: BI usará el índice.
    # (Recomendación: en producción, conservar siempre una clave de negocio en el dataset de scoring)
    churn_scoring_df, churn_global_drivers = score_dataset(
        df_ml_ready=clientes_ml,
        model_obj=churn_model_obj,
        target_candidates=CHURN_TARGET_CANDIDATES,
        id_cols_prefer=["customer_id"],
        threshold=0.50,
        model_name=churn_model_path.stem,
        top_n_drivers=3,
    )

    print("✅ churn_scoring_df generado:", churn_scoring_df.shape)
    print_head(churn_scoring_df.sort_values("score", ascending=False), n=10, title="Top 10 churn scores (mayor riesgo)")
else:
    print("⚠️ No se pudo ejecutar churn scoring (falta modelo o dataset ML-ready).")

In [ ]:
# Export churn outputs
if churn_scoring_df is not None:
    save_csv(churn_scoring_df, OUTPUT_SCORING_DIR / "churn_scoring.csv")
if churn_global_drivers is not None and not churn_global_drivers.empty:
    save_csv(churn_global_drivers, OUTPUT_SCORING_DIR / "churn_global_drivers_top.csv")

## 4) Scoring — Propensión de compra

Este bloque es análogo al churn, pero normalmente el dataset de propensión incluye:
- `customer_id` (ideal)
- `snapshot_date` (fecha de corte)
- features agregadas de la ventana histórica
- label futuro (en training) y **no** en scoring real

Para scoring, si el fichero trae label, lo dejamos solo a efectos de evaluación didáctica.

In [ ]:
# =========================
# 4) Scoring propensión
# =========================

PROP_TARGET_CANDIDATES = [
    "label_will_buy", "will_buy", "label_buy", "buy_next", "target", "y"
]

prop_scoring_df = None
prop_global_drivers = None

if prop_model_path and prop_ml_path.exists():
    prop_model_obj = load_model(prop_model_path)
    prop_ml = safe_read_csv(prop_ml_path, dtype="str")

    prop_scoring_df, prop_global_drivers = score_dataset(
        df_ml_ready=prop_ml,
        model_obj=prop_model_obj,
        target_candidates=PROP_TARGET_CANDIDATES,
        id_cols_prefer=["customer_id", "snapshot_date"],
        threshold=0.50,
        model_name=prop_model_path.stem,
        top_n_drivers=3,
    )

    print("✅ prop_scoring_df generado:", prop_scoring_df.shape)
    print_head(prop_scoring_df.sort_values("score", ascending=False), n=10, title="Top 10 propensión scores")
else:
    print("⚠️ No se pudo ejecutar scoring de propensión (falta modelo o dataset ML-ready).")

In [ ]:
# Export propensión outputs
if prop_scoring_df is not None:
    save_csv(prop_scoring_df, OUTPUT_SCORING_DIR / "propension_scoring.csv")
if prop_global_drivers is not None and not prop_global_drivers.empty:
    save_csv(prop_global_drivers, OUTPUT_SCORING_DIR / "propension_global_drivers_top.csv")

## 5) Diccionario de acciones para BI (ejemplo didáctico)

Generamos una tabla simple para que BI pueda “traducir” deciles a acciones:
- churn: decil 10 = riesgo máximo (acción proactiva)
- propensión: decil 10 = alta probabilidad de compra (campañas de upsell/cross-sell)

> Ajusta el texto al contexto del curso.

In [ ]:
# =========================
# 5) Diccionario de acciones por decil
# =========================

def build_actions_dictionary(problem: str) -> pd.DataFrame:
    rows = []
    for decil in range(1, 11):
        if problem == "churn":
            if decil >= 9:
                action = "Contacto proactivo + oferta retención (alto riesgo)"
                priority = "Alta"
            elif decil >= 6:
                action = "Re-engagement (email/app) + incentivo ligero (riesgo medio)"
                priority = "Media"
            else:
                action = "Mantener comunicación estándar (riesgo bajo)"
                priority = "Baja"
        elif problem == "propension":
            if decil >= 9:
                action = "Campaña upsell/cross-sell (alta probabilidad compra)"
                priority = "Alta"
            elif decil >= 6:
                action = "Recordatorio/remarketing suave (probabilidad media)"
                priority = "Media"
            else:
                action = "No impactar / awareness (probabilidad baja)"
                priority = "Baja"
        else:
            action = "N/A"
            priority = "N/A"
        rows.append({"problem": problem, "decil": decil, "priority": priority, "recommended_action": action})
    return pd.DataFrame(rows)

actions_df = pd.concat([
    build_actions_dictionary("churn"),
    build_actions_dictionary("propension")
], ignore_index=True)

display(actions_df.head(12))
save_csv(actions_df, OUTPUT_SCORING_DIR / "scoring_actions_dictionary.csv")

## 6) Recomendaciones prácticas (formativas)

1. **Evitar leakage:** si tu dataset incluye columnas futuras (p.ej. `label_*` o métricas calculadas después del snapshot), deben excluirse al entrenar y al hacer scoring real.
2. **Conservar claves de negocio:** para BI, es fundamental mantener `customer_id`, `product_id`, etc. En este notebook intentamos recuperarlas, pero lo ideal es que el pipeline las preserve.
3. **Guardar metadatos del entrenamiento:** lista de columnas, umbral, fecha, versión del modelo, etc. Así se evita el “desalineado” de features.
4. **Explicabilidad:** aquí usamos explicaciones sencillas (coeficientes / importancias). Para producción, puedes explorar técnicas como SHAP.